# **ANN Model with BiLSTM**

In [1]:


import pandas as pd
import numpy as np
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from imblearn.over_sampling import SMOTE
from tensorflow.keras.layers import Input, Flatten, SpatialDropout1D, Bidirectional, Reshape
from sklearn.metrics import accuracy_score, classification_report





# Load the dataset
df = pd.read_csv('/content/cleaned_dataset (1).csv')

# Display the first few rows to verify
print("Original DataFrame:")
print(df.head())
 # Use the 'tweet' column for text data
X = df['tweet']
# Use the 'class' column for labels
y = df['class']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
# Handle missing values in 'tweet' column
df['tweet'] = df['tweet'].fillna('')  # Replace NaN with empty strings



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
# Use TfidfVectorizer for the 'tweet_tokens' column
tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(df['tweet'])

# Encode target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Handle class imbalance by converting labels to one-hot encoding
y_categorical = to_categorical(y_encoded)

Original DataFrame:
   Unnamed: 0  count  hate_speech  offensive_language  neither  class  \
0           0      3            0                   0        3      2   
1           1      3            0                   3        0      1   
2           2      3            0                   3        0      1   
3           3      3            0                   2        1      1   
4           4      6            0                   6        0      1   

                                               tweet  \
0  retweet as a woman you should not complain abo...   
1  retweet boy dats coldtyga down bad for cuffin ...   
2  retweet dawg retweet you ever fuck a bitch and...   
3                     retweet she look like a tranny   
4  retweet the shit you hear about me might be tr...   

                                        tweet_tokens  
0  ['retweet', 'woman', 'complain', 'cleaning', '...  
1  ['retweet', 'boy', 'dat', 'coldtyga', 'bad', '...  
2  ['retweet', 'dawg', 'retweet', 'ever

In [6]:

# Define the ANN model with BiLSTM

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(X_tfidf.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
# Reshape the output of the Dense layer to be compatible with LSTM
model.add(Reshape((1, 32)))  # Assuming 32 is the output dimension of the previous Dense layer
model.add(Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(y_categorical.shape[1], activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, min_delta=0.001)

# Sort the indices of the sparse matrix
X_tfidf.sort_indices()

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_tfidf, y_categorical, test_size=0.2, random_state=42)

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32,
                    validation_data=(X_val, y_val),
                    callbacks=[early_stopping],
                    verbose=1)

# Evaluate the model on the validation set
loss, accuracy = model.evaluate(X_val, y_val)
print(f'Validation accuracy: {accuracy:.3f}')

Epoch 1/10
620/620 [==============================] - 190s 293ms/step - loss: 0.4212 - accuracy: 0.8512 - val_loss: 0.3267 - val_accuracy: 0.8816
Epoch 2/10
620/620 [==============================] - 178s 287ms/step - loss: 0.2114 - accuracy: 0.9241 - val_loss: 0.3659 - val_accuracy: 0.8771
Epoch 3/10
620/620 [==============================] - 195s 314ms/step - loss: 0.1115 - accuracy: 0.9603 - val_loss: 0.4540 - val_accuracy: 0.8662
Epoch 4/10
620/620 [==============================] - 177s 285ms/step - loss: 0.0573 - accuracy: 0.9787 - val_loss: 0.5896 - val_accuracy: 0.8731
Epoch 5/10
620/620 [==============================] - 180s 290ms/step - loss: 0.0346 - accuracy: 0.9884 - val_loss: 0.5954 - val_accuracy: 0.8701
Epoch 6/10
155/155 [==============================] - 1s 6ms/step - loss: 0.6637 - accuracy: 0.8731
Validation accuracy: 0.873


In [7]:
# Predict probabilities on the validation set
y_pred_probs = model.predict(X_val)

# Get predicted labels (class with highest probability)
y_pred = np.argmax(y_pred_probs, axis=1)

# Inverse transform encoded true labels to original form
y_val_original = np.argmax(y_val, axis=1)

155/155 [==============================] - 1s 5ms/step


In [8]:
# Extracting metrics separately
print("Accuracy:", accuracy_score(y_val_original, y_pred))
print("Classification Report:\n", classification_report(y_val_original, y_pred))

accuracy = accuracy_score(y_val_original, y_pred)

classification_report_dict = classification_report(y_val_original, y_pred, output_dict=True)
precision = classification_report_dict['weighted avg']['precision']
recall = classification_report_dict['weighted avg']['recall']
f1_score = classification_report_dict['weighted avg']['f1-score']


Accuracy: 0.8731087351220497
Classification Report:
               precision    recall  f1-score   support

           0       0.34      0.16      0.22       290
           1       0.91      0.94      0.93      3832
           2       0.78      0.81      0.80       835

    accuracy                           0.87      4957
   macro avg       0.68      0.64      0.65      4957
weighted avg       0.86      0.87      0.86      4957

